In [ ]:
from PIL import Image
import numpy as np
import os
import hashlib
if os.name == "nt":
    import msvcrt
else:
    import tty
    import termios
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2 as cv
import random
from math import log
import math
import pandas as pd
# from google.colab.patches import cv2_imshow
from tqdm import tqdm
import image_tools as it

## 策略1
分四块，每块选一个像素组成一个2*2的矩阵，放入新的置乱img中，左->右 上->下
左上块  左->右 上->下

右上块  上->下 右->左

左下块  下->上 左->右

右下块  右->左 下->上

In [57]:
def substitution_1(img):
    x, y, _ = img.shape
    # print(x/2 * y/2)
    s_img = img.copy()
    lt_x = 0
    lt_y = 0
    rt_x = 0
    rt_y = y-1
    lb_x = x-1
    lb_y = 0
    rb_x = x-1
    rb_y = y-1
    curr_x = 0
    curr_y = 0
    for i in range(int(x/2 * y/2)):
        s_img[curr_x][curr_y] = img[lt_x][lt_y]
        s_img[curr_x][curr_y+1] = img[rt_x][rt_y]
        s_img[curr_x+1][curr_y] = img[lb_x][lb_y]
        s_img[curr_x+1][curr_y+1] = img[rb_x][rb_y]
        if lt_y == y/2 - 1:
            lt_x += 1
            lt_y = 0
        else:
            lt_y += 1
        if rt_x == x/2 - 1:
            rt_x = 0
            rt_y -= 1
        else:
            rt_x += 1
        if rb_y == y/2:
            rb_x -= 1
            rb_y = y-1
        else:
            rb_y -= 1
        if lb_x == x/2:
            lb_x = x-1
            lb_y += 1
        else:
            lb_x -= 1
        if curr_y+2 == y:
            curr_x += 2
            curr_y = 0
        else:
            curr_y += 2
    print(lt_x,' ',lt_y)
    print(rt_x,' ',rt_y)
    return s_img


def reverse_substitution_1(img):
    rs_img = img.copy()
    x, y, _ = img.shape
    lt_x = 0
    lt_y = 0
    rt_x = 0
    rt_y = y-1
    lb_x = x-1
    lb_y = 0
    rb_x = x-1
    rb_y = y-1
    curr_x = 0
    curr_y = 0
    for i in range(int(x/2 * y/2)):
        rs_img[lt_x][lt_y] = img[curr_x][curr_y]
        rs_img[rt_x][rt_y] = img[curr_x][curr_y+1]
        rs_img[lb_x][lb_y] = img[curr_x+1][curr_y]
        rs_img[rb_x][rb_y] = img[curr_x+1][curr_y+1]
        if lt_y == y/2 - 1:
            lt_x += 1
            lt_y = 0
        else:
            lt_y += 1
        if rt_x == x/2 - 1:
            rt_x = 0
            rt_y -= 1
        else:
            rt_x += 1
        if rb_y == y/2:
            rb_x -= 1
            rb_y = y-1
        else:
            rb_y -= 1
        if lb_x == x/2:
            lb_x = x-1
            lb_y += 1
        else:
            lb_x -= 1
        if curr_y+2 == y:
            curr_x += 2
            curr_y = 0
            # print(curr_x,' ',curr_y)
        else:
            curr_y += 2
    return rs_img


In [78]:
img = cv.imread("graphs\s-box\orig.png")
# print(img[0][249])
simg1 = substitution_1(img)
ssimg1 = substitution_1(simg1)
sssimg1 = substitution_1(ssimg1)
# print(simg1[0][1])
cv.imwrite('graphs\s-box\orig_s_1.png',simg1)

# cv.imshow('sssimg1',sssimg1)
# cv.waitKey(0)
rsimg1 = reverse_substitution_1(simg1)
cv.imwrite('graphs\s-box\orig_rs_1.png',rsimg1)
# cv.imshow('12',rsimg1)
# cv.waitKey(0)
# cv.destroyAllWindows()
print(rsimg1[0][249])
it.assert_img_equal('graphs\s-box\orig.png','graphs\s-box\orig_rs_1.png',1)



# 经历若干次s-box效果较好
simg = img.copy()
for i in range(10):
    simg = substitution_1(simg)
cv.imshow('simg 10 times',simg)
cv.waitKey(0)

rsimg = simg.copy()
for i in range(10):
    rsimg = reverse_substitution_1(rsimg)
cv.imshow('rsimg 10 times',rsimg)
cv.waitKey(0)
# rs1 = reverse_substitution_1(sssimg1)
# rs2 = reverse_substitution_1(rs1)
# rs3 = reverse_substitution_1(rs2)
# cv.imshow('rs3',rs3)
# cv.waitKey(0)

125   0
0   124
125   0
0   124
125   0
0   124
[ 44  63 100]
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124
125   0
0   124


-1

## 策略2
分四块，每块选一个像素组成一个2*2的矩阵，放入新的置乱img中，左->右 上->下
左上块  左->右 上->下

右上块  上->下 右->左

左下块  下->上 左->右

右下块  右->左 下->上

In [67]:
def substitution_2(img):
    x, y, _ = img.shape
    # print(x/2 * y/2)
    s_img = img.copy()
    px, py = 0, 0  # 遍历img
    curr_x = 0
    curr_y = 0
    for i in range(int(x/2 * y/2)):
        s_img[curr_x][curr_y] = img[px][py]
        s_img[curr_x][curr_y+1] = img[px][py+int(y/2)]
        s_img[curr_x+1][curr_y] = img[px+int(x/2)][py]
        s_img[curr_x+1][curr_y+1] = img[px+int(x/2)][py+int(y/2)]
        if py == y/2-1:
            px += 1
            py = 0
        else:
            py += 1
        if curr_y+2 == y:
            curr_x += 2
            curr_y = 0
        else:
            curr_y += 2
    return s_img


def reverse_substitution_2(img):
    rs_img = img
    return


125   0
0   124
[ 44  63 100]


0

In [72]:
img = cv.imread("graphs\s-box\orig.png")
# print(img[0][249])
simg2 = substitution_2(img)
ssimg2 = substitution_2(simg2)
sssimg2 = substitution_2(ssimg2)
# print(simg1[0][1])
cv.imwrite('graphs\s-box\orig_s_2.png',simg2)
cv.imshow('simg',sssimg2)
cv.waitKey(0)
cv.destroyAllWindows()